In [33]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import requests
import os
import pyodbc
from datetime import datetime
from sqlalchemy import create_engine, exc
from dotenv import load_dotenv

load_dotenv()

True

In [34]:
start_date = '2023-01-01'
end_date = '2024-06-30'
#forex_pair = 'EURUSD'
crypto = 'DOGEUSD'

In [35]:

def polygon(ticker, date_start, date_end):
    # Set up the GET request
    polygon_url = 'https://api.polygon.io/v2/aggs/ticker'
    #adjusted_ticker = f'/C:{forex_ticker}'
    adjusted_ticker = f'/X:{ticker}'
    timespan = '/range/1/month'
    date_from = f'/{date_start}'
    date_to = f'/{date_end}'
    params = '?adjusted=true&sort=asc&'
    polygon_key = os.environ['polygon_api_key']

    params = {'adjusted':'true', 
            'sort': 'asc',
            'apiKey': polygon_key
            }

    full_url = polygon_url + adjusted_ticker + timespan + date_from + date_to

    # Request the data
    response2 = requests.get(full_url, params)
    polygon_data = response2.json()

    # Access the "results" part of the response
    results = polygon_data["results"]
    
    df_polygon = pd.DataFrame(results)
    df_polygon.head(5)

    # Convert the Unix timestamp to datetime
    df_polygon['date'] = pd.to_datetime(df_polygon['t'], unit='ms')

    # Format the datetime to the desired string format (YYYY-MM-DD)
    df_polygon['date'] = df_polygon['date'].dt.strftime('%Y-%m-%d')

    # Drop the unix time column
    df_polygon = df_polygon.drop(columns=['t'])

    # Rename cols
    df_polygon.rename(columns={'v': 'volume', 'o':'open', 'c':'close', 'h':'high', 'l':'low'}, inplace=True)

    #df_polygon = df_polygon[['date', 'open', 'high', 'low', 'close', 'volume', 'vw', 'n']]
    df_polygon = df_polygon[['date', 'open', 'high', 'low', 'close']]
    
    # Convert date from object to datetime
    df_polygon[['date']] = df_polygon[['date']].apply(pd.to_datetime)
    df_polygon['date'] = df_polygon['date'].dt.strftime('%Y-%m')
    df_polygon['date'] = df_polygon['date'] + '-01'
    
    #df_polygon['volume'] = df_polygon['volume'].astype(int)
    
    return df_polygon


In [36]:
# Check for nulls
def null_check(df):
    null_counts = df.isnull().sum()
    return(null_counts)

In [37]:
df_1 = polygon(crypto, start_date, end_date)

print(null_check(df_1))

df_1['symbol'] = crypto

df_1

date     0
open     0
high     0
low      0
close    0
dtype: int64


,date,open,high,low,close,symbol
0,2023-01-01,0.070200,0.097810,0.068990,0.096110,DOGEUSD
1,2023-02-01,0.096120,0.099810,0.078373,0.080767,DOGEUSD
2,2023-03-01,0.080860,0.083100,0.062530,0.076980,DOGEUSD
3,2023-04-01,0.076960,0.105197,0.074591,0.079510,DOGEUSD
4,2023-05-01,0.079510,0.081602,0.068466,0.071670,DOGEUSD
5,2023-06-01,0.071680,0.073350,0.053547,0.066430,DOGEUSD
6,2023-07-01,0.066553,0.083840,0.063330,0.077850,DOGEUSD
7,2023-08-01,0.077860,0.078382,0.055000,0.063764,DOGEUSD
8,2023-09-01,0.063780,0.064950,0.059199,0.062080,DOGEUSD
9,2023-10-01,0.062080,0.075239,0.056887,0.068280,DOGEUSD


In [38]:
# Define the connection string for Windows Authentication
server = 'MoemenLaptop'
database = 'InvestmentPortfolio'
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Establish the connection
try:
    conn = pyodbc.connect(connection_string)
    print("Connection successful!")
    cursor = conn.cursor()

    # Iterate over rows of the DataFrame
    for index, row in df_1.iterrows():
        # Example SQL insert command
        sql_command = "INSERT INTO Monthly_Data (date, [open], high, low, [close], symbol) VALUES (?, ?, ?, ?, ?, ?)"
        
        # Execute the command
        cursor.execute(sql_command, tuple(row))

    # Commit the transaction
    conn.commit()
    
    print("Records Inserted!")

except Exception as e:
    print(f"Error: {e}")
    
    
conn.close()

Connection successful!
Records Inserted!
